In [1]:
# File to create the desired complete database to use in the models later
# Make extra table with mrn_csn_pairs of all patients we are currently including
import numpy as np
import pandas as pd
import sqlite3
import tsfresh
import dask.dataframe as dd

In [2]:
# Get path and connection
path_in = "S:\Dehydration_stroke\Team Emerald\Working Data\Preprocessed\Working\Merged.db"
path_out = "S:\Dehydration_stroke\Team Emerald\Working Data\Preprocessed\Working\Models.db"
con = sqlite3.connect(path_in)
con_out = sqlite3.connect(path_out)

In [3]:
cursor = con.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('static_predictors',), ('outcomes',), ('timeseries_instantaneous',), ('timeseries_startstop',)]


In [4]:
insheet = pd.read_sql_query("SELECT * FROM timeseries_instantaneous", con)
startstop = pd.read_sql_query("SELECT * FROM timeseries_startstop", con)

In [8]:
insheet = insheet.append(startstop)

In [9]:
# Get all unique patitents so we can pull first 24 hours of data
pats = insheet.sort_values('mrn_csn_pair')['mrn_csn_pair'].unique()

In [10]:
first = insheet[(insheet['timestamp'] < 1440) & (insheet['timestamp'] >= 0)]

In [11]:
first = first.sort_values('mrn_csn_pair').reset_index(drop=True)

In [12]:
# Have to break dataset up to avoid memory issues, takes ~20 min to run
extracted_flowsheet = pd.DataFrame()
for i in range(8):
    split1 = int(pats.shape[0] / 8) * i
    split2 = int(pats.shape[0] / 8) * (i + 1)
    key1 = pats[split1]
    key2 = pats[split2]
    index1 = first[first['mrn_csn_pair'] == key1].index[0]
    index2 = first[first['mrn_csn_pair'] == key2].index[0] - 1
    if i == 7:
        index2 = first.shape[0] - 1
    temp = first.loc[index1 : index2, :]
    extracted_temp = tsfresh.extract_features(temp, column_id='mrn_csn_pair', column_sort='timestamp', column_kind='measure', column_value='value', n_jobs=6)
    extracted_flowsheet = extracted_flowsheet.append(extracted_temp, ignore_index=True)
    

Feature Extraction: 100%|██████████████████████████████████████████████████████████████| 30/30 [01:31<00:00,  3.07s/it]


In [13]:
extracted_flowsheet = extracted_flowsheet.dropna(axis=1, how='all')

In [14]:
# Use tsfresh to do some impution to remove NaN values, its ok if some are filled with 0's as these will be removed in feature selection
tsfresh.utilities.dataframe_functions.impute(extracted_flowsheet)

C:\Users\ahepp1\Anaconda3\lib\site-packages\tsfresh\utilities\dataframe_functions.py:171: RuntimeWarning: The columns ['NRBC__sample_entropy' 'cvc_line__sample_entropy' 'drain__sample_entropy'
 'line__sample_entropy' 'anxiolytic__sample_entropy'] did not have any finite values. Filling with zeros.
  warnings.warn("The columns {} did not have any finite values. Filling with zeros.".format(


,ALBUMIN__variance_larger_than_standard_deviation,ALBUMIN__has_duplicate_max,ALBUMIN__has_duplicate_min,ALBUMIN__has_duplicate,ALBUMIN__sum_values,ALBUMIN__abs_energy,ALBUMIN__mean_abs_change,ALBUMIN__mean_change,ALBUMIN__mean_second_derivative_central,ALBUMIN__median,...,picc_line__ratio_beyond_r_sigma__r_6,picc_line__ratio_beyond_r_sigma__r_7,picc_line__ratio_beyond_r_sigma__r_10,picc_line__count_above__t_0,picc_line__count_below__t_0,picc_line__lempel_ziv_complexity__bins_2,picc_line__lempel_ziv_complexity__bins_3,picc_line__lempel_ziv_complexity__bins_5,picc_line__lempel_ziv_complexity__bins_10,picc_line__lempel_ziv_complexity__bins_100
0,0.0,0.0,0.0,0.0,3.9,15.21,0.3,-0.266667,0.1,3.9,...,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,0.0,0.0,0.0,0.0,4.2,16.81,0.3,-0.266667,0.1,3.7,...,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,0.0,0.0,0.0,0.0,2.6,6.76,0.3,-0.266667,0.1,2.6,...,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,0.0,0.0,0.0,0.0,4.2,17.64,0.3,-0.266667,0.1,4.2,...,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,0.0,0.0,0.0,0.0,4.4,19.36,0.3,-0.266667,0.1,4.4,...,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2096,0.0,0.0,0.0,0.0,5.1,26.01,0.3,-0.266667,0.1,5.1,...,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2097,0.0,0.0,0.0,0.0,4.2,16.81,0.3,-0.266667,0.1,3.7,...,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2098,0.0,0.0,0.0,0.0,4.2,16.81,0.3,-0.266667,0.1,3.7,...,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2099,0.0,0.0,0.0,0.0,4.2,16.81,0.3,-0.266667,0.1,3.7,...,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [15]:
extracted_flowsheet.insert(0, 'mrn_csn_pair', pats)

In [16]:
first.to_sql('flowsheet_first24h', con_out, if_exists='replace')

In [17]:
# Uhh..have to save to excel because sql reallllyyy hates lots of columns.
extracted_flowsheet.to_csv('extracted_flowsheet_first24h.csv')

In [18]:
con.close()
con_out.close()